# How to Use: `calc_humbl_channel`


## Log into OpenBB

In [ ]:
%load_ext autoreload
%autoreload 2

from openbb import obb

from humbldata.core.utils.env import Env
from humbldata.core.utils.openbb_helpers import obb_login

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)


In [ ]:
import yfinance as yf

yf.__version__

## Collect Data

In [ ]:
from humbldata.toolbox.technical.humbl_channel.helpers import add_window_index
from humbldata.toolbox.toolbox_helpers import log_returns

symbols = ["AAPL", "PCT", "NVDA", "SMCI", "TSLA", "AMD", "SPY", "GLD", "BTC-USD", "ETH-USD"]
rr_list = ["MSFT", "AAPL", "AMZN", "META", "GOOGL", "NFLX", "TSLA", "NVDA"]
sector_symbols = [
    "XLF",
    "XLU",
    "XLK",
    "XLE",
    "XLI",
    "XLV",
    "XLY",
    "XLP",
    "XLB",
    "XLRE",
    "XTL"
]
simple= ["AAPL", "AMZN", "AMD"]


data = (
    obb.equity.price.historical(
        "MP",
        provider="yfinance",
        start_date="2024-01-01",
        end_date="2025-04-16",
        adjustment="splits_and_dividends",
    ).to_polars()
)

data = log_returns(data, _column_name="close")


In [ ]:
data

In [9]:
from openbb_core.provider.utils.helpers import get_requests_session
import yfinance as yf

session = get_requests_session()
print(session.headers)
print(session.cookies)
print(session.proxies)

{'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:86.1) Gecko/20100101 Firefox/86.1', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}
<RequestsCookieJar[]>
{}


## Calculate Mandelbrot Channel


In [14]:
from humbldata.toolbox.technical.humbl_channel.model import calc_humbl_channel


mandelbrot = calc_humbl_channel(
    data,
    window="1m",
    rv_adjustment=True,
    rv_method="yz",
    rv_grouped_mean=False,
    rs_method="RS",
    live_price=True
).collect()


In [ ]:
mandelbrot

## Find Prices Near the Top/Bottom of the Channel

In [ ]:
mandelbrot_historical.filter(
    (pl.col("close") >= pl.col("top_price") * 0.975)
    & (pl.col("close") <= pl.col("bottom_price") * 1.025)
)
